In [1]:
import torch

In [39]:
labels = torch.randint(low=-1, high=2, size=(2, 3), dtype=torch.float32)
preds = torch.tanh(labels) - torch.rand_like(labels) * 0.1
preds, labels

(tensor([[-0.0895, -0.7782, -0.0077],
         [ 0.7273, -0.7636, -0.0463]]),
 tensor([[ 0., -1.,  0.],
         [ 1., -1.,  0.]]))

In [40]:
from torch.nn import BCELoss, MSELoss

bce = BCELoss(reduction='mean')
mse = MSELoss(reduction='mean')

In [41]:
abs_preds = torch.abs(preds)
abs_labels = torch.abs(labels)
abs_preds, abs_labels

(tensor([[0.0895, 0.7782, 0.0077],
         [0.7273, 0.7636, 0.0463]]),
 tensor([[0., 1., 0.],
         [1., 1., 0.]]))

In [43]:
# Get the indices where the abs_labels are 1
pos_mask_indices = abs_labels.nonzero(as_tuple=True)
pos_mask_indices, abs_labels[pos_mask_indices], abs_preds[pos_mask_indices]

((tensor([0, 1, 1]), tensor([1, 0, 1])),
 tensor([1., 1., 1.]),
 tensor([0.7782, 0.7273, 0.7636]))

In [44]:
loss = bce(abs_preds, abs_labels) + mse(preds[pos_mask_indices], labels[pos_mask_indices])

In [45]:
loss

tensor(0.2244)

In [47]:
def compute_mask_loss(preds: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
    bce = BCELoss(reduction='mean')
    mse = MSELoss(reduction='mean')

    abs_preds = torch.abs(preds)
    abs_labels = torch.abs(labels)

    # Get the indices where the abs_labels are 1
    pos_mask_indices = abs_labels.nonzero(as_tuple=True)

    bce_term = bce(abs_preds, abs_labels)
    mse_term = mse(preds[pos_mask_indices], labels[pos_mask_indices])

    return bce_term + mse_term

In [48]:
compute_mask_loss(preds, labels)

tensor(0.2244)

In [50]:
torch.sum(torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))

tensor(55)

In [56]:
a = torch.stack([torch.tensor(1.0), torch.tensor(2.0)], dim=0)
torch.mean(a)

tensor(1.5000)

In [63]:
import pandas as pd

picke_path = "../data/epic-kitchens-100-annotations/EPIC_100_train.pkl"
df = pd.read_pickle(picke_path)

df["start_ts_td"] = pd.to_timedelta(df["start_timestamp"])
df["stop_ts_td"] = pd.to_timedelta(df["stop_timestamp"])
df["duration_in_s"] = (df["stop_ts_td"] - df["start_ts_td"]).dt.total_seconds()
del df["start_ts_td"]
del df["stop_ts_td"]

df.head()

,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes,duration_in_s
narration_id,,,,,,,,,,,,,,,
P01_01_0,P01,P01_01,00:00:01.089,00:00:00.14,00:00:03.37,8,202,open door,open,3,door,3,[door],[3],3.23
P01_01_1,P01,P01_01,00:00:02.629,00:00:04.37,00:00:06.17,262,370,turn on light,turn-on,6,light,114,[light],[114],1.80
P01_01_10,P01,P01_01,00:00:23.340,00:00:24.97,00:00:26.20,1498,1572,open drawer,open,3,drawer,8,[drawer],[8],1.23
P01_01_100,P01,P01_01,00:07:57.919,00:07:59.75,00:08:00.88,28785,28852,take cup,take,0,cup,13,[cup],[13],1.13
P01_01_101,P01,P01_01,00:08:00.020,00:08:01.47,00:08:02.21,28888,28932,open cupboard,open,3,cupboard,3,[cupboard],[3],0.74


count    67217.000000
mean         3.118102
std          5.396898
min          0.200000
25%          0.970000
50%          1.570000
75%          3.130000
max        297.150000
Name: duration_in_s, dtype: float64

In [67]:
ids = [4, 10, 11]

df[df.verb_class.isin(ids) & df.duration_in_s < 2.0]

,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes,duration_in_s
narration_id,,,,,,,,,,,,,,,
P01_01_0,P01,P01_01,00:00:01.089,00:00:00.14,00:00:03.37,8,202,open door,open,3,door,3,[door],[3],3.23
P01_01_1,P01,P01_01,00:00:02.629,00:00:04.37,00:00:06.17,262,370,turn on light,turn-on,6,light,114,[light],[114],1.80
P01_01_10,P01,P01_01,00:00:23.340,00:00:24.97,00:00:26.20,1498,1572,open drawer,open,3,drawer,8,[drawer],[8],1.23
P01_01_100,P01,P01_01,00:07:57.919,00:07:59.75,00:08:00.88,28785,28852,take cup,take,0,cup,13,[cup],[13],1.13
P01_01_101,P01,P01_01,00:08:00.020,00:08:01.47,00:08:02.21,28888,28932,open cupboard,open,3,cupboard,3,[cupboard],[3],0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P37_103_71,P37,P37_103,00:06:16.903,00:06:17.17,00:06:17.67,18858,18883,turn off tap,turn-off,8,tap,0,[tap],[0],0.50
P37_103_72,P37,P37_103,00:06:22.154,00:06:17.86,00:06:23.77,18893,19188,take pan,take,0,pan,5,[pan],[5],5.91
P37_103_73,P37,P37_103,00:06:26.404,00:06:23.45,00:06:32.66,19172,19633,pour out boiled water,pour-out,9,water:boiled,27,[water:boiled],[27],9.21
